In [1]:
import numpy as np
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
from machine_learning.SVM_defopt import SVMOptimizer
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from opt_funct.data_fuct import STRATEGY_XGBOOST_BUY
from opt_funct.data_fuct import STRATEGY_REGLOG_BUY
from opt_funct.data_fuct import STRATEGY_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_SVC_BUY
from opt_funct.data_fuct import STRATEGY_REGLOG_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SVC_BUY
from opt_funct.data_fuct import STRATEGY_XGBOOST_SELL
from opt_funct.data_fuct import STRATEGY_REGLOG_SELL
from opt_funct.data_fuct import STRATEGY_SVC_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_SVC_SELL
from opt_funct.data_fuct import STRATEGY_REGLOG_SVC_SELL
from opt_funct.data_fuct import STRATEGY_XGBOOST_REGLOG_SVC_SELL
import optuna
from opt_funct.data_fuct import Operation



In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_5m_train.csv'))
df

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


**Entrenamos nuestros modelos para compra y venta**

In [3]:
# Buy
close_data = df[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train_buy, x_test_buy, y_train_buy, y_test_buy = train_test_split(x, y, test_size=0.3, shuffle=False)

close_data2 = df[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_SELL']].dropna()
x = close_data2.drop(columns=['Y_SELL'])
y = close_data2[['Y_SELL']]
x_train_sell, x_test_sell, y_train_sell, y_test_sell = train_test_split(x, y, test_size=0.3, shuffle=False)

In [4]:
# Escalado para el conjunto de compra
scaler_buy = StandardScaler()
x_train_buy_scaled = scaler_buy.fit_transform(x_train_buy)
x_test_buy_scaled = scaler_buy.transform(x_test_buy)

# Escalado para el conjunto de venta
scaler_sell = StandardScaler()
x_train_sell_scaled = scaler_sell.fit_transform(x_train_sell)
x_test_sell_scaled = scaler_sell.transform(x_test_sell)

In [7]:
# buy
xgboost_optimizer = XGBOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
xgboost_best_params_buy, best_model_xgboost_buy = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
reg_log_best_params_buy, best_model_reg_log_buy = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_buy_scaled, y_train_buy, x_test_buy_scaled, y_test_buy)
SVC_best_params_buy, best_model_SVC_buy = SVC_optimizer.svm_optuna()

# sell
xgboost_optimizer = XGBOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
xgboost_best_params_sell, best_model_xgboost_sell = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
reg_log_best_params_sell, best_model_reg_log_sell = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_sell_scaled, y_train_sell, x_test_sell_scaled, y_test_sell)
SVC_best_params_sell, best_model_SVC_sell = SVC_optimizer.svm_optuna()

[I 2024-03-09 19:04:50,122] A new study created in memory with name: no-name-2e222307-e683-40bd-9016-816c92a81bc9
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:04:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-09 19:04:50,169] Trial 0 finished with value: 0.678879670360265 and parameters: {'n_estimators': 14, 'max_depth': 11, 'max_leaves': 65, 'learning_rate': 0.6613635401477155, 'booster': 'gblinear', 'gamma': 36.22897378706918, 'reg_alpha': 8.588477054030976, 'reg_lambda': 6.659685245474945}. Best is trial 0 with value: 0.678879670360265.
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:04:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-0

F1: 0.678879670360265
Best hyperparameters: {'n_estimators': 14, 'max_depth': 11, 'max_leaves': 65, 'learning_rate': 0.6613635401477155, 'booster': 'gblinear', 'gamma': 36.22897378706918, 'reg_alpha': 8.588477054030976, 'reg_lambda': 6.659685245474945}
Execution time: 0.17911348740259805 minutes


[I 2024-03-09 19:05:19,913] Trial 0 finished with value: 0.678879670360265 and parameters: {'C': 9.00489091918956, 'fit_intercept': False, 'l1_ratio': 0.3822485270472161}. Best is trial 0 with value: 0.678879670360265.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-09 19:05:39,121] Trial 1 finished with value: 0.678879670360265 and parameters: {'C': 9.61226185632867, 'fit_intercept': False, 'l1_ratio': 0.7467829015058945}. Best is trial 0 with value: 0.678879670360265.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-0

F1 Score: 0.678879670360265
Best hyperparameters: {'C': 9.00489091918956, 'fit_intercept': False, 'l1_ratio': 0.3822485270472161}
Execution time: 3.1412082354227704 minutes


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 19:08:23,818] Trial 0 finished with value: 0.5768978819419172 and parameters: {'C': 4.420853103399049, 'kernel': 'rbf', 'degree': 5, 'gamma': 4.0}. Best is trial 0 with value: 0.5768978819419172.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 19:08:29,999] Trial 1 finished with value: 

...

F1: 0.4660319807745633
Best hyperparameters: {'n_estimators': 104, 'max_depth': 37, 'max_leaves': 73, 'learning_rate': 0.32725866685110555, 'booster': 'dart', 'gamma': 8.860625442542869, 'reg_alpha': 8.639634765389056, 'reg_lambda': 6.6483464185123555}
Execution time: 0.18575068712234497 minutes


[I 2024-03-09 19:10:20,663] A new study created in memory with name: no-name-f3f37506-d6db-48cb-8d5f-a95fddaeb15e
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-09 19:10:20,782] Trial 0 finished with value: 0.0 and parameters: {'C': 0.7601284170779663, 'fit_intercept': True, 'l1_ratio': 0.8931276175110809}. Best is trial 0 with value: 0.0.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-09 19:10:20,949] Trial 1 finished with value: 0.0 and parameters: {'C': 2.0427652134261303, 'fit_intercept': True, 'l1_ratio': 0.8568

F1 Score: 0.0
Best hyperparameters: {'C': 0.7601284170779663, 'fit_intercept': True, 'l1_ratio': 0.8931276175110809}
Execution time: 0.024464452266693117 minutes


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 19:10:27,442] Trial 0 finished with value: 0.5918867050493019 and parameters: {'C': 5.248463653769732, 'kernel': 'poly', 'degree': 2, 'gamma': 1.0}. Best is trial 0 with value: 0.5918867050493019.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 19:10:34,864] Trial 1 finished with value:

F1 Score: 0.6245083579154376
Best hyperparameters: {'C': 2.6641545065835452, 'kernel': 'poly', 'degree': 2, 'gamma': 'scale'}
Execution time: 1.3648560682932536 minutes


In [8]:
# Indicadores Buy
close_data['xg_boost_buy'] = best_model_xgboost_buy.predict(x)
close_data['reg_log_buy'] = best_model_reg_log_buy.predict(x)
close_data['SVC_buy'] = best_model_SVC_buy.predict(x)
# Indicadores Sell
close_data = close_data.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3'])
close_data['xg_boost_sell'] = best_model_xgboost_sell.predict(x)
close_data['reg_log_sell'] = best_model_reg_log_sell.predict(x)
close_data['SVC_sell'] = best_model_SVC_sell.predict(x)
# Convertimos a boleanos
close_data['xg_boost_buy'] = np.where(close_data['xg_boost_buy']==1, True, False)
close_data['xg_boost_sell'] = np.where(close_data['xg_boost_sell']==1, True, False)
close_data

/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,Timestamp,Close,xg_boost_buy,reg_log_buy,SVC_buy,xg_boost_sell,reg_log_sell,SVC_sell
27,1609778700,129.413299,True,True,True,False,False,True
28,1609779000,129.148498,True,True,True,False,False,True
29,1609779300,128.820007,True,True,True,False,False,True
30,1609779600,128.580001,True,True,True,False,False,True
31,1609779900,128.351699,True,True,True,False,False,True
...,...,...,...,...,...,...,...,...
39574,1672432800,129.054992,True,True,True,False,False,True
39575,1672433100,129.380004,True,True,True,False,False,True
39576,1672433400,129.764404,True,True,True,False,False,True
39577,1672433700,129.949996,True,True,True,False,False,True


**Mostramos nuestras combinaciones posibles de compra y venta**

In [6]:
indicadores_compra = ['xg_boost_buy', 'reg_log_buy' , 'SVC_buy']
indicadores_venta = ['xg_boost_sell', 'reg_log_sell' , 'SVC_sell']

combinaciones_indicadores_compra = []
combinaciones_indicadores_venta = []

for i in range(1, len(indicadores_compra) + 1):
    combinaciones_indicadores_compra.extend(combinations(indicadores_compra, i))

for i in range(1, len(indicadores_venta) + 1):
    combinaciones_indicadores_venta.extend(combinations(indicadores_venta, i))
    
combinaciones_indicadores_compra, combinaciones_indicadores_venta

([('xg_boost_buy',),
  ('reg_log_buy',),
  ('SVC_buy',),
  ('xg_boost_buy', 'reg_log_buy'),
  ('xg_boost_buy', 'SVC_buy'),
  ('reg_log_buy', 'SVC_buy'),
  ('xg_boost_buy', 'reg_log_buy', 'SVC_buy')],
 [('xg_boost_sell',),
  ('reg_log_sell',),
  ('SVC_sell',),
  ('xg_boost_sell', 'reg_log_sell'),
  ('xg_boost_sell', 'SVC_sell'),
  ('reg_log_sell', 'SVC_sell'),
  ('xg_boost_sell', 'reg_log_sell', 'SVC_sell')])

In [7]:
cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]


Calculamos la mejor combinacion para compra

In [22]:
best_global_strategy = {'name': None, 'value': float('-inf')}

def optimize(trial):
    # Definición de los parámetros a optimizar
    stop_loss_long = trial.suggest_float('stop_loss_long', 0.01, 0.3)
    take_profit_long = trial.suggest_float('take_profit_long', 0.01, 0.3)
    n_shares = trial.suggest_int('n_shares', 10, 100)
    
    # Inicialización de la estrategia con los parámetros actuales
    xg_boost_buy = STRATEGY_XGBOOST_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_buy = STRATEGY_REGLOG_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    svc_buy = STRATEGY_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_buy = STRATEGY_XGBOOST_REGLOG_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_svc_buy = STRATEGY_XGBOOST_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_svc_buy = STRATEGY_REGLOG_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_svc_buy = STRATEGY_XGBOOST_REGLOG_SVC_BUY(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_long=stop_loss_long,
        take_profit_long=take_profit_long
    )
    
    strategy_values = {
        'xgboost_buy': xg_boost_buy.run_strategy(),
        'reglog_buy': reglog_buy.run_strategy(),
        'svc_buy': svc_buy.run_strategy(),
        'xgboost_reglog_buy': xgboost_reglog_buy.run_strategy(),
        'xgboost_svc_buy': xgboost_svc_buy.run_strategy(),
        'reglog_svc_buy': reglog_svc_buy.run_strategy(),
        'xgboost_reglog_svc_buy': xgboost_reglog_svc_buy.run_strategy()
    }

    # Encuentra el mayor valor de estrategia y su correspondiente estrategia
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_strategy['value']:
        best_global_strategy['name'] = best_strategy_name
        best_global_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [23]:
# Inicializar y ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(optimize, n_trials=5, n_jobs=1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_strategy['name']
best_strategy_value = best_global_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best buy overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best buy strategy parameters:", best_params)

[I 2024-03-10 12:49:47,057] A new study created in memory with name: no-name-e5b12d4d-aee1-4dd6-b08a-4ec1e97e6afa
[I 2024-03-10 12:51:50,423] Trial 0 finished with value: 994752.2598888731 and parameters: {'stop_loss_long': 0.2608110348613971, 'take_profit_long': 0.2313586740579268, 'stop_loss_short': 0.23832273590569034, 'take_profit_short': 0.10263154096611177, 'n_shares': 66}. Best is trial 0 with value: 994752.2598888731.


values: {'xgboost_buy': 994752.2598888731, 'reglog_buy': 994752.2598888731, 'svc_buy': 994752.2598888731, 'xgboost_reglog_buy': 994752.2598888731, 'xgboost_svc_buy': 994752.2598888731, 'reglog_svc_buy': 994752.2598888731, 'xgboost_reglog_svc_buy': 994752.2598888731}
Best buy strategy: xgboost_buy with value: 994752.2598888731


[I 2024-03-10 12:53:28,360] Trial 1 finished with value: 954801.0805850024 and parameters: {'stop_loss_long': 0.24375062098984013, 'take_profit_long': 0.021342456667383256, 'stop_loss_short': 0.23213886915059787, 'take_profit_short': 0.1497213628463973, 'n_shares': 80}. Best is trial 0 with value: 994752.2598888731.


values: {'xgboost_buy': 954801.0805850024, 'reglog_buy': 954801.0805850024, 'svc_buy': 954801.0805850024, 'xgboost_reglog_buy': 954801.0805850024, 'xgboost_svc_buy': 954801.0805850024, 'reglog_svc_buy': 954801.0805850024, 'xgboost_reglog_svc_buy': 954801.0805850024}
Best buy strategy: xgboost_buy with value: 954801.0805850024


[I 2024-03-10 12:54:56,309] Trial 2 finished with value: 990869.8886498918 and parameters: {'stop_loss_long': 0.15611643171133044, 'take_profit_long': 0.03577418003359859, 'stop_loss_short': 0.2081067850233701, 'take_profit_short': 0.16815316922026163, 'n_shares': 93}. Best is trial 0 with value: 994752.2598888731.


values: {'xgboost_buy': 990869.8886498918, 'reglog_buy': 990869.8886498918, 'svc_buy': 990869.8886498918, 'xgboost_reglog_buy': 990869.8886498918, 'xgboost_svc_buy': 990869.8886498918, 'reglog_svc_buy': 990869.8886498918, 'xgboost_reglog_svc_buy': 990869.8886498918}
Best buy strategy: xgboost_buy with value: 990869.8886498918


[I 2024-03-10 12:56:26,959] Trial 3 finished with value: 994039.0291575851 and parameters: {'stop_loss_long': 0.05603276823062428, 'take_profit_long': 0.24156472453186667, 'stop_loss_short': 0.10060379515139466, 'take_profit_short': 0.20844095326979953, 'n_shares': 91}. Best is trial 0 with value: 994752.2598888731.


values: {'xgboost_buy': 994039.0291575851, 'reglog_buy': 994039.0291575851, 'svc_buy': 994039.0291575851, 'xgboost_reglog_buy': 994039.0291575851, 'xgboost_svc_buy': 994039.0291575851, 'reglog_svc_buy': 994039.0291575851, 'xgboost_reglog_svc_buy': 994039.0291575851}
Best buy strategy: xgboost_buy with value: 994039.0291575851


[I 2024-03-10 13:01:48,657] Trial 4 finished with value: 953329.6986370742 and parameters: {'stop_loss_long': 0.07806974311254251, 'take_profit_long': 0.02200900353757675, 'stop_loss_short': 0.01734605096503308, 'take_profit_short': 0.12596664502092825, 'n_shares': 23}. Best is trial 0 with value: 994752.2598888731.


values: {'xgboost_buy': 953329.6986370742, 'reglog_buy': 953329.6986370742, 'svc_buy': 953329.6986370742, 'xgboost_reglog_buy': 953329.6986370742, 'xgboost_svc_buy': 953329.6986370742, 'reglog_svc_buy': 953329.6986370742, 'xgboost_reglog_svc_buy': 953329.6986370742}
Best buy strategy: xgboost_buy with value: 953329.6986370742
Best overall strategy: xgboost_buy with value: 994752.2598888731
Best parameters: {'stop_loss_long': 0.2608110348613971, 'take_profit_long': 0.2313586740579268, 'stop_loss_short': 0.23832273590569034, 'take_profit_short': 0.10263154096611177, 'n_shares': 66}


Calculamos la mejor combinacion para venta

In [25]:
best_global_sell_strategy = {'name': None, 'value': float('-inf')}

def optimize_sell(trial):
    # Definición de los parámetros a optimizar
    stop_loss_short = trial.suggest_float('stop_loss_short', 0.01, 0.3)
    take_profit_short = trial.suggest_float('take_profit_short', 0.01, 0.3)
    n_shares = trial.suggest_int('n_shares', 10, 100)
    
    # Inicialización de la estrategia con los parámetros actuales
    xg_boost_sell = STRATEGY_XGBOOST_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_sell = STRATEGY_REGLOG_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    svc_sell = STRATEGY_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_sell = STRATEGY_XGBOOST_REGLOG_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_svc_sell = STRATEGY_XGBOOST_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    reglog_svc_sell = STRATEGY_REGLOG_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    # Inicialización de la estrategia con los parámetros actuales
    xgboost_reglog_svc_sell = STRATEGY_XGBOOST_REGLOG_SVC_SELL(
        df=close_data,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss_short=stop_loss_short,
        take_profit_short=take_profit_short
    )
    
    strategy_values = {
        'xgboost_sell': xg_boost_sell.run_strategy(),
        'reglog_sell': reglog_sell.run_strategy(),
        'svc_sell': svc_sell.run_strategy(),
        'xgboost_reglog_sell': xgboost_reglog_sell.run_strategy(),
        'xgboost_svc_sell': xgboost_svc_sell.run_strategy(),
        'reglog_svc_sell': reglog_svc_sell.run_strategy(),
        'xgboost_reglog_svc_sell': xgboost_reglog_svc_sell.run_strategy()
    }

    # Encuentra el mayor valor de estrategia y su correspondiente estrategia
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_sell_strategy['value']:
        best_global_sell_strategy['name'] = best_strategy_name
        best_global_sell_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [ ]:
# Inicializar y ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(optimize_sell, n_trials=5, n_jobs=1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_sell_strategy['name']
best_strategy_value = best_global_sell_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best sell overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best sell strategy parameters:", best_params)

[I 2024-03-10 13:15:52,218] A new study created in memory with name: no-name-b96c6c07-449a-434e-83a9-1d4118673470
[I 2024-03-10 13:48:48,220] Trial 0 finished with value: 378576203.4586934 and parameters: {'stop_loss_short': 0.21563280501991727, 'take_profit_short': 0.09793135531440177, 'n_shares': 67}. Best is trial 0 with value: 378576203.4586934.
